# Varying diameter

In this notebook we will try to create a scaling law to account for a varying propeller diameter as well as a varying number of blade and a varying altitude. Twist law should also vary but for now it will be left as is.

In [ ]:
import warnings

warnings.filterwarnings(action="ignore")

We'll adjust slightly the defined set here to take something similar to the previous notebook, that is $\pi_2 = \frac{picth}{PITCH_{REF}}$ and $\pi_3 = n_{blade}$

In [ ]:
import pandas as pd
import numpy as np

results_fixed_pitch = pd.read_csv("data/doe_fixed_pitch.csv")
results_ICE = pd.read_csv("data/doe_ICE.csv")
results_TP = pd.read_csv("data/doe_TP.csv")

results = pd.concat([results_fixed_pitch, results_ICE, results_TP])

number_of_element = len(results["Thrust"].to_numpy())

print("Number of element pre screening: ", number_of_element)

sampling = 25

efficiency_screening = np.where(results["Efficiency"].to_numpy() >= 0.5)

thrust_array = results["Thrust"].to_numpy()[efficiency_screening][0::sampling]
power_array = results["Power"].to_numpy()[efficiency_screening][0::sampling]
eta_array = results["Efficiency"].to_numpy()[efficiency_screening][0::sampling]
sigma_array = results["Solidity"].to_numpy()[efficiency_screening][0::sampling]
AF_array = results["Activity Factor"].to_numpy()[efficiency_screening][0::sampling]
lambda_array = results["Aspect ratio"].to_numpy()[efficiency_screening][0::sampling]
n_rot_val = results["Rotational speed"].to_numpy()[efficiency_screening][0::sampling]
D_val = results["Diameter"].to_numpy()[efficiency_screening][0::sampling]
n_blade_val = results["Number of blades"].to_numpy()[efficiency_screening][0::sampling]
RHO_val = results["Density"].to_numpy()[efficiency_screening][0::sampling]
pitch_val = results["Pitch at 0.75R"].to_numpy()[efficiency_screening][0::sampling]
PITCH_REF_val = results["Ref Pitch at 0.75R"].to_numpy()[efficiency_screening][0::sampling]
v_val = results["Flight speed"].to_numpy()[efficiency_screening][0::sampling]
altitude = results["Altitude"].to_numpy()[efficiency_screening][0::sampling]
twist_blade = results["Blade twist between root and tip"].to_numpy()[efficiency_screening][
    0::sampling
]

print(np.min(n_rot_val), np.max(n_rot_val))
print(np.min(D_val), np.max(D_val))
print(np.min(RHO_val), np.max(RHO_val))
print(np.min(v_val), np.max(v_val))
print(np.min(n_blade_val), np.max(n_blade_val))

print("Number of element post screening: ", len(n_rot_val))

ct_array = thrust_array / (RHO_val * n_rot_val**2 * D_val**4)
cp_array = power_array / (RHO_val * n_rot_val**3 * D_val**5)

# Trying $\eta$ as a function of Ct and J

In [ ]:
# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

import pandas as pd

from stdatm import Atmosphere

atm = Atmosphere(altitude, altitude_in_feet=False)

pi1 = eta_array
pi2 = ct_array
pi3 = v_val / (n_rot_val * D_val)
pi4 = (
    v_val**2.0 + (n_rot_val * 2.0 * np.pi) ** 2.0 * (D_val / 2.0) ** 2.0
) / atm.speed_of_sound**2.0
pi5 = v_val * D_val / atm.kinematic_viscosity
pi6 = sigma_array
pi7 = AF_array
pi8 = lambda_array
doe_eta = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7, pi8]
doe_eta = pd.DataFrame(doe_eta, columns=["pi1", "pi2", "pi3", "pi4", "pi5", "pi6", "pi7", "pi8"])

# Fit with 3rd order power-law model the obtained Pi DOE
model = regression_models(
    doe_eta.values, elected_pi0="pi1", order=2, log_space=True, ymax_axis=10, test_mode=True
)
perform_regression(doe_eta.values, model, choosen_model=20)

# Trying Ct as a function of J, twist and other parameters

In [ ]:
# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression
from stdatm import Atmosphere

atm = Atmosphere(altitude, altitude_in_feet=False)

pi1 = ct_array
pi2 = v_val / (n_rot_val * D_val)
pi3 = (
    v_val**2.0 + (n_rot_val * 2.0 * np.pi) ** 2.0 * (D_val / 2.0) ** 2.0
) / atm.speed_of_sound**2.0
pi4 = v_val * D_val / atm.kinematic_viscosity
pi5 = sigma_array
pi6 = pitch_val / PITCH_REF_val
pi7 = AF_array
pi8 = twist_blade
doe_ct = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7, pi8]
doe_ct = pd.DataFrame(
    doe_ct, columns=["Ct", "J", "M_tip", "Re", "sigma", "Twist", "AF", "Twist_blade"]
)

In [ ]:
# Fit with 3rd order power-law model the obtained Pi DOE
model = regression_models(
    doe_ct.values, elected_pi0="pi1", order=4, log_space=True, ymax_axis=10, test_mode=True
)
perform_regression(doe_ct.values, model, choosen_model=30)

In [ ]:
print(np.round(ct_array[:5], 3))

In [ ]:
print(np.min(pi2), np.max(pi2))
print(np.min(pi3), np.max(pi3))
print(np.min(pi4 / 1e6), np.max(pi4 / 1e6))
print(np.min(pi5), np.max(pi5))
print(np.min(pitch_val / PITCH_REF_val), np.max(pitch_val / PITCH_REF_val))
print(np.min(pi7), np.max(pi7))
print(np.min(pi8), np.max(pi8))

# Trying Ct as a function of J, Cp and other parameters

In [ ]:
# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

from stdatm import Atmosphere

atm = Atmosphere(altitude, altitude_in_feet=False)

pi1 = ct_array
pi2 = v_val / (n_rot_val * D_val)
pi3 = (
    v_val**2.0 + (n_rot_val * 2.0 * np.pi) ** 2.0 * (D_val / 2.0) ** 2.0
) / atm.speed_of_sound**2.0
pi4 = v_val * D_val / atm.kinematic_viscosity
pi5 = sigma_array
pi6 = cp_array
pi7 = AF_array
pi8 = twist_blade
doe_ct = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7, pi8]
doe_ct = pd.DataFrame(
    doe_ct, columns=["Ct", "J", "M_tip", "Re_D", "sigma", "Cp", "AF", "Twist_blade"]
)

In [ ]:
models, axs, fig = regression_models(
    doe_ct.values,
    elected_pi0="pi1",
    order=4,
    log_space=True,
    test_mode=True,
    return_axes=True,
    skip_cross_validation=True,
)

In [ ]:
max_error = models["max |e|"]
mean_error = models["ave. |e|"]
average_rel_error = models["ave. e"]
standard_deviation = models["sigma e"]

In [ ]:
import plotly.graph_objects as go

fig_plotly = go.Figure()
scatter = go.Scatter(
    x=np.arange(len(max_error[0])), y=np.clip(max_error[0], 0, 100), mode="lines", name="Max error"
)
fig_plotly.add_trace(scatter)
fig_plotly.add_hline(y=5.0)
fig_plotly.show()

In [ ]:
import plotly.graph_objects as go

fig_plotly_2 = go.Figure()
scatter = go.Scatter(
    x=np.arange(len(mean_error[0])),
    y=np.clip(mean_error[0], 0, 100),
    mode="lines",
    name="Mean error",
)
fig_plotly_2.add_trace(scatter)
fig_plotly_2.add_hline(y=5.0)
fig_plotly_2.show()

In [ ]:
import plotly.graph_objects as go

fig_plotly_3 = go.Figure()
scatter = go.Scatter(
    x=np.arange(len(standard_deviation[0])),
    y=np.clip(standard_deviation[0], 0, 100),
    mode="lines",
    name="Standard deviation",
)
fig_plotly_3.add_trace(scatter)
fig_plotly_3.add_hline(y=5.0)
fig_plotly_3.show()

In [ ]:
# Fit with 3rd order power-law model the obtained Pi DOE
perform_regression(doe_ct.values, models, chosen_model=64)

In [ ]:
idx = np.where(altitude == 0.0)[0]
print(np.unique(D_val))

# Trying Cp as a function of J, Ct and other parameters

In [ ]:
# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

from stdatm import Atmosphere

atm = Atmosphere(altitude, altitude_in_feet=False)

pi1 = cp_array
pi2 = v_val / (n_rot_val * D_val)
pi3 = (
    v_val**2.0 + (n_rot_val * 2.0 * np.pi) ** 2.0 * (D_val / 2.0) ** 2.0
) / atm.speed_of_sound**2.0
pi4 = v_val * D_val / atm.kinematic_viscosity  # Somehow reduced the deviation :O
pi5 = sigma_array
pi6 = ct_array
pi7 = AF_array
pi8 = twist_blade
doe_cp = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7, pi8]
doe_cp = pd.DataFrame(
    doe_cp, columns=["Cp", "J", "M_tip", "Re_D", "sigma", "Ct", "AF", "Twist_blade"]
)

In [ ]:
# Fit with 3rd order power-law model the obtained Pi DOE
models, axs, fig = regression_models(
    doe_cp.values,
    elected_pi0="pi1",
    order=4,
    log_space=True,
    test_mode=True,
    return_axes=True,
    skip_cross_validation=True,
)

In [ ]:
max_error = models["max |e|"]
mean_error = models["ave. |e|"]
average_rel_error = models["ave. e"]
standard_deviation = models["sigma e"]

In [ ]:
import plotly.graph_objects as go

fig_plotly = go.Figure()
scatter = go.Scatter(
    x=np.arange(len(max_error[0])), y=np.clip(max_error[0], 0, 100), mode="lines", name="Max error"
)
fig_plotly.add_trace(scatter)
fig_plotly.add_hline(y=5.0)
fig_plotly.show()

In [ ]:
import plotly.graph_objects as go

fig_plotly_2 = go.Figure()
scatter = go.Scatter(
    x=np.arange(len(mean_error[0])),
    y=np.clip(mean_error[0], 0, 100),
    mode="lines",
    name="Mean error",
)
fig_plotly_2.add_trace(scatter)
fig_plotly_2.add_hline(y=5.0)
fig_plotly_2.show()

In [ ]:
import plotly.graph_objects as go

fig_plotly_3 = go.Figure()
scatter = go.Scatter(
    x=np.arange(len(standard_deviation[0])),
    y=np.clip(standard_deviation[0], 0, 100),
    mode="lines",
    name="Standard deviation",
)
fig_plotly_3.add_trace(scatter)
fig_plotly_3.add_hline(y=5.0)
fig_plotly_3.show()

In [ ]:
# Fit with 3rd order power-law model the obtained Pi DOE
perform_regression(doe_cp.values, models, chosen_model=23)

In [ ]:
print(np.min(pi1), np.max(pi1))
print(np.min(pi2), np.max(pi2))
print(np.min(pi3), np.max(pi3))
print(np.min(pi4), np.max(pi4))
print(np.min(pi5), np.max(pi5))
print(np.min(pi6), np.max(pi6))
print(np.min(pi7), np.max(pi7))
print(np.min(pi8), np.max(pi8))

In [ ]:
def cp_from_ct(j, tip_mach, re_d, solidity, ct, activity_factor, twist_blade):
    cp = (
        10**0.97966
        * j
        ** (
            -0.01288 * np.log10(re_d)
            + 0.06626 * np.log10(j) * np.log10(activity_factor) ** 2
            + 0.12256 * np.log10(solidity) * np.log10(ct) * np.log10(activity_factor)
            + 0.27017 * np.log10(ct) * np.log10(activity_factor) * np.log10(twist_blade)
            + 0.08618 * np.log10(solidity) * np.log10(twist_blade) ** 2
            - 0.01302 * np.log10(j) ** 2 * np.log10(re_d)
            - 0.51762 * np.log10(solidity) * np.log10(activity_factor) * np.log10(twist_blade)
            + 0.06004 * np.log10(ct) ** 3
            + 0.71752
            + 0.10064 * np.log10(re_d) * np.log10(solidity) * np.log10(twist_blade)
        )
        * tip_mach ** (-0.00087 * np.log10(re_d) ** 2 * np.log10(twist_blade))
        * re_d
        ** (
            +0.02225 * np.log10(solidity) * np.log10(ct) ** 2
            + 0.00060 * np.log10(re_d) ** 3
            + 0.00190 * np.log10(re_d) * np.log10(ct) ** 2
            - 0.00613 * np.log10(re_d) ** 2
            + 0.02932 * np.log10(ct) ** 2
        )
        * solidity ** (-0.02670 * np.log10(solidity) ** 2 * np.log10(activity_factor))
        * ct
        ** (
            +1.45291
            + 0.01386 * np.log10(ct) ** 2 * np.log10(activity_factor)
            + 0.00362 * np.log10(ct) * np.log10(twist_blade) ** 2
            - 0.05156 * np.log10(ct) ** 2
        )
        * activity_factor ** (-0.00472 * np.log10(activity_factor) ** 2 * np.log10(twist_blade))
    )

    return cp

# Trying Cp as a function of J, Ct and other parameters (New Reynolds)

In [ ]:
# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

from stdatm import Atmosphere

atm = Atmosphere(altitude, altitude_in_feet=False)

pi1 = cp_array
pi2 = v_val / (n_rot_val * D_val)
pi3 = (
    v_val**2.0 + (n_rot_val * 2.0 * np.pi) ** 2.0 * (D_val / 2.0) ** 2.0
) / atm.speed_of_sound**2.0
pi4 = (
    np.sqrt(v_val**2.0 + (n_rot_val * 2.0 * np.pi) ** 2.0 * (D_val / 2.0) ** 2.0)
    * D_val
    / atm.kinematic_viscosity
)  # Somehow reduced the deviation :O
pi5 = sigma_array
pi6 = ct_array
pi7 = AF_array
pi8 = twist_blade
doe_cp = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7, pi8]
doe_cp = pd.DataFrame(
    doe_cp, columns=["Cp", "J", "M_tip", "Re_D", "sigma", "Ct", "AF", "Twist_blade"]
)

# Fit with 3rd order power-law model the obtained Pi DOE
models, axs, fig = regression_models(
    doe_cp.values,
    elected_pi0="pi1",
    order=4,
    log_space=True,
    test_mode=True,
    return_axes=True,
    skip_cross_validation=True,
)

In [ ]:
import plotly.graph_objects as go

max_error = models["max |e|"]
mean_error = models["ave. |e|"]
average_rel_error = models["ave. e"]
standard_deviation = models["sigma e"]

fig_plotly_2 = go.Figure()
scatter = go.Scatter(
    x=np.arange(len(mean_error[0])),
    y=np.clip(mean_error[0], 0, 100),
    mode="lines",
    name="Mean error",
)
fig_plotly_2.add_trace(scatter)
fig_plotly_2.add_hline(y=5.0)
fig_plotly_2.show()

fig_plotly_3 = go.Figure()
scatter = go.Scatter(
    x=np.arange(len(standard_deviation[0])),
    y=np.clip(standard_deviation[0], 0, 100),
    mode="lines",
    name="Standard deviation",
)
fig_plotly_3.add_trace(scatter)
fig_plotly_3.add_hline(y=5.0)
fig_plotly_3.show()

In [ ]:
# Fit with 3rd order power-law model the obtained Pi DOE
perform_regression(doe_cp.values, models, chosen_model=21)

In [ ]:
def cp_from_ct_new(j, tip_mach, re_d, solidity, ct, activity_factor, twist_blade):
    # Careful, new definition of the tip mach which should now be computed using the absolute
    # speed on the blades

    cp = (
        10**2.31538
        * j
        ** (
            +0.05414 * np.log10(re_d)
            + 0.06795 * np.log10(j) * np.log10(activity_factor) ** 2
            - 0.47030 * np.log10(ct)
            - 0.05973 * np.log10(j) ** 2
            + 0.17931 * np.log10(ct) * np.log10(activity_factor) * np.log10(twist_blade)
            - 0.19079 * np.log10(twist_blade) ** 2
            - 0.07782 * np.log10(j) * np.log10(activity_factor) * np.log10(twist_blade)
            - 0.07683 * np.log10(ct) ** 2 * np.log10(activity_factor)
        )
        * tip_mach ** (+0.00011 * np.log10(re_d) ** 3)
        * re_d
        ** (
            +0.02100 * np.log10(solidity) * np.log10(ct) ** 2
            - 0.32454
            - 0.00078 * np.log10(ct) ** 2 * np.log10(twist_blade)
            + 0.00371 * np.log10(re_d) * np.log10(ct) ** 2
            + 0.00015 * np.log10(re_d) ** 3
        )
        * solidity ** (-0.02698 * np.log10(solidity) ** 2 * np.log10(activity_factor))
        * ct
        ** (
            +1.48237
            + 0.01548 * np.log10(ct) ** 2 * np.log10(activity_factor)
            + 0.10762 * np.log10(ct)
            - 0.05141 * np.log10(ct) ** 2
        )
        * activity_factor ** (-0.00292 * np.log10(activity_factor) ** 2 * np.log10(twist_blade))
    )

    return cp

In [ ]:
def cp_from_ct_new_new(j, tip_mach, re_d, solidity, ct, activity_factor, twist_blade):
    # Careful, new definition of the tip mach which should now be computed using the absolute
    # speed on the blades

    cp = (
        10**2.43553
        * j
        ** (
            0.61554
            + 0.06980 * np.log10(j) * np.log10(ct) * np.log10(twist_blade)
            - 0.01794 * np.log10(re_d) * np.log10(ct) * np.log10(activity_factor)
            + 0.02595 * np.log10(tip_mach) * np.log10(solidity) * np.log10(ct)
            + 0.00430 * np.log10(j) * np.log10(re_d) ** 2.0
            + 0.09827 * np.log10(ct) * np.log10(twist_blade)
            + 0.03663 * np.log10(solidity) * np.log10(activity_factor)
        )
        * tip_mach ** (-0.00097 * np.log10(re_d) ** 2.0 * np.log10(tip_mach))
        * re_d
        ** (
            -0.35804
            + 0.00018 * np.log10(re_d) ** 3.0
            - 0.01879 * np.log10(ct) ** 3.0
            + 0.00119 * np.log10(re_d) * np.log10(solidity) * np.log10(activity_factor)
            - 0.00886 * np.log10(ct) ** 2.0 * np.log10(twist_blade)
        )
        * solidity
        ** (
            0.08015 * np.log10(ct) ** 2.0 * np.log10(activity_factor)
            + 0.04562 * np.log10(solidity) * np.log10(activity_factor) ** 2.0
            - 0.04121 * np.log10(solidity) * np.log10(ct) * np.log10(twist_blade)
        )
        * ct
        ** (
            1.33164
            + 0.06989 * np.log10(ct) * np.log10(activity_factor)
            + 0.00206 * np.log10(ct) * np.log10(twist_blade) ** 2.0
            + 0.03617 * np.log10(ct) ** 2.0 * np.log10(activity_factor)
        )
    )

    return cp

In [ ]:
import plotly.graph_objects as go

from stdatm import Atmosphere

efficiency_screening = np.where(results["Efficiency"].to_numpy() >= 0.5)

thrust_array_verif = results["Thrust"].to_numpy()[efficiency_screening]
power_array_verif = results["Power"].to_numpy()[efficiency_screening]
eta_array_verif = results["Efficiency"].to_numpy()[efficiency_screening]
sigma_array_verif = results["Solidity"].to_numpy()[efficiency_screening]
AF_array_verif = results["Activity Factor"].to_numpy()[efficiency_screening]
lambda_array_verif = results["Aspect ratio"].to_numpy()[efficiency_screening]
n_rot_val_verif = results["Rotational speed"].to_numpy()[efficiency_screening]
D_val_verif = results["Diameter"].to_numpy()[efficiency_screening]
n_blade_val_verif = results["Number of blades"].to_numpy()[efficiency_screening]
RHO_val_verif = results["Density"].to_numpy()[efficiency_screening]
pitch_val_verif = results["Pitch at 0.75R"].to_numpy()[efficiency_screening]
PITCH_REF_val_verif = results["Ref Pitch at 0.75R"].to_numpy()[efficiency_screening]
v_val_verif = results["Flight speed"].to_numpy()[efficiency_screening]
altitude_verif = results["Altitude"].to_numpy()[efficiency_screening]
twist_blade_verif = results["Blade twist between root and tip"].to_numpy()[efficiency_screening]

ct_array_verif = thrust_array_verif / (RHO_val_verif * n_rot_val_verif**2 * D_val_verif**4)
cp_array_verif = power_array_verif / (RHO_val_verif * n_rot_val_verif**3 * D_val_verif**5)

atm_verif = Atmosphere(altitude_verif, altitude_in_feet=False)

pi1_verif = cp_array_verif
pi2_verif = v_val_verif / (n_rot_val_verif * D_val_verif)
pi3_verif = (
    v_val_verif**2.0 + (n_rot_val_verif * 2.0 * np.pi) ** 2.0 * (D_val_verif / 2.0) ** 2.0
) / atm_verif.speed_of_sound**2.0
pi4_verif = (
    np.sqrt(v_val_verif**2.0 + (n_rot_val_verif * 2.0 * np.pi) ** 2.0 * (D_val_verif / 2.0) ** 2.0)
    * D_val_verif
    / atm_verif.kinematic_viscosity
)  # Somehow reduced the deviation :O
pi5_verif = sigma_array_verif
pi6_verif = ct_array_verif
pi7_verif = AF_array_verif
pi8_verif = twist_blade_verif

print(np.min(pi1_verif), np.max(pi1_verif))
print(np.min(pi2_verif), np.max(pi2_verif))
print(np.min(pi3_verif), np.max(pi3_verif))
print(np.min(pi4_verif), np.max(pi4_verif))
print(np.min(pi5_verif), np.max(pi5_verif))
print(np.min(pi6_verif), np.max(pi6_verif))
print(np.min(pi7_verif), np.max(pi7_verif))
print(np.min(pi8_verif), np.max(pi8_verif))

pi1_computed = cp_from_ct_new_new(
    pi2_verif, pi3_verif, pi4_verif, pi5_verif, pi6_verif, pi7_verif, pi8_verif
)

fig_verif = go.Figure()
scatter_vs_computed = go.Scatter(
    x=pi1_verif,
    y=pi1_computed,
    mode="markers",
    marker=dict(
        color="gray",
        size=10,
        # line=dict(
        #     color='black',
        #     width=1
        # )
    ),
)

print("Mean absolute error on dataset")
mean_absolute_error = round(np.mean(np.abs(pi1_computed - pi1_verif) / pi1_verif) * 100, 3)
print(mean_absolute_error, "%")
print("Mean error on dataset")
mean_error = round(np.mean((pi1_computed - pi1_verif) / pi1_verif * 100), 3)
print(mean_error, "%")
print("Mean deviation on dataset")
standard_deviation = round(np.std((pi1_computed - pi1_verif) / pi1_verif * 100), 3)
print(standard_deviation, "%")

fig_verif.add_trace(scatter_vs_computed)
scatter_id = go.Scatter(
    x=pi1_verif, y=pi1_verif, mode="lines", line=dict(color="black", width=4, dash="dash")
)
fig_verif.add_trace(scatter_id)
fig_verif.update_layout(
    title_text="Comparaison of formula on validation set",
    title_x=0.5,
    xaxis_title=r"$C_P$",
    yaxis_title=r"$C_P^{*}=f(C_T,J, M_{tip}, Re_D, \sigma, AF, \theta_{blade})$",
    font_size=15,
    width=1200,
    height=800,
)
fig_verif.update_layout(showlegend=False)
fig_verif.update_layout(
    annotations=[
        go.layout.Annotation(
            text=" Mean error = "
            + str(mean_error)
            + " % <br>"
            + " Mean absolute error = "
            + str(mean_absolute_error)
            + " % <br>"
            + " Standard deviation = "
            + str(standard_deviation)
            + " % ",
            align="left",
            font=dict(size=20),
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0.03,
            y=0.97,
            bordercolor="black",
            borderwidth=1,
            bgcolor="white",
        )
    ]
)
fig_verif.show()

# fig_verif.write_image("validation.pdf")

idx_first_wrong = np.where(np.abs(pi1_verif - 0.01578974) < 1e-7)[0]
idx_second_wrong = np.where(np.abs(pi1_verif - 1.28047) < 1e-7)[0]

print("Absolute parameters")
print("Thrust", thrust_array_verif[idx_second_wrong])
print("Power", power_array_verif[idx_second_wrong])
print("Efficiency", eta_array_verif[idx_second_wrong])
print("Solidity", sigma_array_verif[idx_second_wrong])
print("Activity Factor", AF_array_verif[idx_second_wrong])
print("Aspect ratio", lambda_array_verif[idx_second_wrong])
print("Rotation speed", n_rot_val_verif[idx_second_wrong])
print("Diameter", D_val_verif[idx_second_wrong])
print("N_blade", n_blade_val_verif[idx_second_wrong])
print("Airspeed", v_val_verif[idx_second_wrong])

print("\nPies")
print("Cp", cp_array_verif[idx_second_wrong])
print("J", pi2_verif[idx_second_wrong])
print("M_tip", pi3_verif[idx_second_wrong])
print("Re_D", pi4_verif[idx_second_wrong])
print("Solidity", sigma_array_verif[idx_second_wrong])
print("Ct", pi6_verif[idx_second_wrong])
print("AF", pi7_verif[idx_second_wrong])
print("Twist blade", twist_blade_verif[idx_second_wrong])

In [ ]:
import plotly.graph_objects as go

from stdatm import Atmosphere

thrust_array_verif = results["Thrust"].to_numpy()
power_array_verif = results["Power"].to_numpy()
eta_array_verif = results["Efficiency"].to_numpy()
sigma_array_verif = results["Solidity"].to_numpy()
AF_array_verif = results["Activity Factor"].to_numpy()
lambda_array_verif = results["Aspect ratio"].to_numpy()
n_rot_val_verif = results["Rotational speed"].to_numpy()
D_val_verif = results["Diameter"].to_numpy()
n_blade_val_verif = results["Number of blades"].to_numpy()
RHO_val_verif = results["Density"].to_numpy()
pitch_val_verif = results["Pitch at 0.75R"].to_numpy()
PITCH_REF_val_verif = results["Ref Pitch at 0.75R"].to_numpy()
v_val_verif = results["Flight speed"].to_numpy()
altitude_verif = results["Altitude"].to_numpy()
twist_blade_verif = results["Blade twist between root and tip"].to_numpy()

ct_array_verif = thrust_array_verif / (RHO_val_verif * n_rot_val_verif**2 * D_val_verif**4)
cp_array_verif = power_array_verif / (RHO_val_verif * n_rot_val_verif**3 * D_val_verif**5)

atm_verif = Atmosphere(altitude_verif, altitude_in_feet=False)

pi1_verif = cp_array_verif
pi2_verif = v_val_verif / (n_rot_val_verif * D_val_verif)
pi3_verif = (
    v_val_verif**2.0 + (n_rot_val_verif * 2.0 * np.pi) ** 2.0 * (D_val_verif / 2.0) ** 2.0
) / atm_verif.speed_of_sound**2.0
pi4_verif = (
    np.sqrt(v_val_verif**2.0 + (n_rot_val_verif * 2.0 * np.pi) ** 2.0 * (D_val_verif / 2.0) ** 2.0)
    * D_val_verif
    / atm_verif.kinematic_viscosity
)  # Somehow reduced the deviation :O
pi5_verif = sigma_array_verif
pi6_verif = ct_array_verif
pi7_verif = AF_array_verif
pi8_verif = twist_blade_verif

pi1_computed = cp_from_ct_new_new(
    pi2_verif, pi3_verif, pi4_verif, pi5_verif, pi6_verif, pi7_verif, pi8_verif
)

fig_verif = go.Figure()
scatter_vs_computed = go.Scatter(
    x=pi1_verif,
    y=pi1_computed,
    mode="markers",
    name="Computed",
)
fig_verif.add_trace(scatter_vs_computed)
scatter_id = go.Scatter(x=pi1_verif, y=pi1_verif, mode="lines", name="Data", line_color="red")
fig_verif.add_trace(scatter_id)
fig_verif.update_layout(
    title_text="Comparaison of formula on full DOE",
    title_x=0.5,
    xaxis_title=r"$\pi_1$",
    yaxis_title=r"$\pi_1=f(\pi_2,\pi_3,\pi_4,\pi_5,\pi_6,\pi_7,\pi_8)$",
    yaxis_range=[-0.1, 1.1 * np.max(pi1_verif)],
    font_size=15,
    width=1200,
    height=800,
)
fig_verif.show()

idx_first_wrong = np.where(np.abs(pi1_verif - 0.01578974) < 1e-7)[0]
idx_second_wrong = np.where(np.abs(pi1_verif - 1.28047) < 1e-7)[0]

print("Absolute parameters")
print("Thrust", thrust_array_verif[idx_second_wrong])
print("Power", power_array_verif[idx_second_wrong])
print("Efficiency", eta_array_verif[idx_second_wrong])
print("Solidity", sigma_array_verif[idx_second_wrong])
print("Activity Factor", AF_array_verif[idx_second_wrong])
print("Aspect ratio", lambda_array_verif[idx_second_wrong])
print("Rotation speed", n_rot_val_verif[idx_second_wrong])
print("Diameter", D_val_verif[idx_second_wrong])
print("N_blade", n_blade_val_verif[idx_second_wrong])
print("Airspeed", v_val_verif[idx_second_wrong])

print("\nPies")
print("Cp", cp_array_verif[idx_second_wrong])
print("J", pi2_verif[idx_second_wrong])
print("M_tip", pi3_verif[idx_second_wrong])
print("Re_D", pi4_verif[idx_second_wrong])
print("Solidity", sigma_array_verif[idx_second_wrong])
print("Ct", pi6_verif[idx_second_wrong])
print("AF", pi7_verif[idx_second_wrong])
print("Twist blade", twist_blade_verif[idx_second_wrong])